Note, still in the process of experimenting with parameters

Import packages

In [1]:
import tensorflow as tf
import numpy as np
#import matplotlib.pyplot as plt
import pandas as pds
from tensorflow import keras
from hyperas import optim
from hyperas.distributions import choice, uniform
from hyperopt import Trials, STATUS_OK, tpe
from tensorflow.keras.optimizers import Adadelta, Nadam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

Using TensorFlow backend.


In [2]:
def create_model(x_train, y_train, x_test, y_test):
    """
    Previous best:
    {'Dense': 2,
     'Dense_1': 4,
     'Dense_2': 0,
     'activation': 1,
     'activation_1': 0,
     'batch_size': 1,
     'optimizer': 1}
    {'Dense': 2,
     'Dense_1': 2,
     'Dense_2': 1,
     'activation': 2,
     'activation_1': 0,
     'activation_2': 1,
     'batch_size': 0,
     'optimizer': 0}
    """
    model = Sequential()
    
    model.add(Dense({{choice([3,19,38])}}, activation={{choice([None, 'relu', 'tanh'])}}, input_shape=(19,)))
    model.add(Dense({{choice([1,19,38])}}, activation={{choice([None, 'relu', 'tanh'])}}))
    model.add(Dense({{choice([1,19,38])}}, activation={{choice([None, 'relu', 'tanh'])}}))
    model.add(Dense(1, activation={{choice([None, 'relu'])}}))

    model.compile(loss='mae', 
                  optimizer={{choice(['Adadelta', 'Adam'])}},
                  metrics=['mae', 'mape']
                 )
    
    #result = model.fit(x_train,
    #                  y_train,
    ##                  epochs = 5,
    #                  batch_size=10,
    #                  validation_data=[x_test, y_test])
    result = model.fit(x_train, y_train,
                   batch_size={{choice([64, 128])}},
                   epochs=3,
                   verbose=1,
                   validation_split=0.1)
    
    loss_metric = np.amax(result.history['mean_absolute_error']) 
    print('Current epoch:', result.history['mean_absolute_error'], result.history['mean_absolute_percentage_error'])
    return {'loss': loss_metric, 'status': STATUS_OK, 'model': model}

In [3]:
def data():
    data = pds.read_pickle('dataset_model2v2_w_label.pkl')
    dataset = data#.iloc[0:10000]
    labels = dataset['label']
    dataset = dataset.drop(columns=['label'])

    num_test_segments = 700
    msk = np.random.randint(1, dataset.journey_number.unique()[-1], num_test_segments)
    mask = dataset['journey_number'].isin(msk)

    train_input = dataset[~mask]
    test_input = dataset[mask]

    train_labels = labels[~mask]
    test_labels = labels[mask]

    train_data = train_input.drop(columns=['journey_number'])
    test_data = test_input.drop(columns=['journey_number'])

    x_train = train_input.drop(columns=['journey_number'])
    x_test = test_data
    y_train = train_labels
    y_test = test_labels
    
    return x_train, y_train, x_test, y_test

In [4]:
best_run, best_model = optim.minimize(model=create_model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=35,
                                      trials=Trials(),
                                      notebook_name="Model2v2HPO"
                                     )



>>> Imports:
#coding=utf-8

try:
    import tensorflow as tf
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import pandas as pds
except:
    pass

try:
    from tensorflow import keras
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from tensorflow.keras.optimizers import Adadelta, Nadam
except:
    pass

try:
    from tensorflow.keras import Sequential
except:
    pass

try:
    from tensorflow.keras.layers import Dense
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Dense': hp.choice('Dense', [3,19,38]),
        'Dense_1': hp.choice('Dense_1', [1,19,38]),
        'activation': hp.choice('activation', [None, 'relu', 'tanh']),
        'Dense_2': hp.choice('Dense_2', [1,19,38]),
        'activation_1': hp.choice('activation_1', [None, 'relu', 'tanh'

Epoch 3/3
2467080/2467080 [==============================] - 36s 15us/step - loss: 11.2919 - mean_absolute_error: 11.2919 - mean_absolute_percentage_error: 10609706.3987 - val_loss: 11.1946 - val_mean_absolute_error: 11.1946 - val_mean_absolute_percentage_error: 14115018.3952
Current epoch: [15.061662024196268, 11.41046281796917, 11.291864028098292] [59750984.43608129, 13670976.538711196, 10609706.39873409]
Train on 2467080 samples, validate on 274121 samples
Epoch 1/3
2467080/2467080 [==============================] - 20s 8us/step - loss: 38.3206 - mean_absolute_error: 38.3206 - mean_absolute_percentage_error: 450825496.3631 - val_loss: 36.5325 - val_mean_absolute_error: 36.5325 - val_mean_absolute_percentage_error: 521470598.2612
Epoch 2/3
2467080/2467080 [==============================] - 19s 8us/step - loss: 19.5097 - mean_absolute_error: 19.5097 - mean_absolute_percentage_error: 110000162.4080 - val_loss: 15.4251 - val_mean_absolute_error: 15.4251 - val_mean_absolute_percentage_er

2467080/2467080 [==============================] - 21s 8us/step - loss: 17.2099 - mean_absolute_error: 17.2099 - mean_absolute_percentage_error: 82788449.6680 - val_loss: 13.8217 - val_mean_absolute_error: 13.8217 - val_mean_absolute_percentage_error: 32433067.7198
Current epoch: [44.52708303270721, 25.414795913855862, 17.209898638929772] [226187778.84886634, 129909856.23134138, 82788449.66797768]
Train on 2467080 samples, validate on 274121 samples
Epoch 1/3
2467080/2467080 [==============================] - 22s 9us/step - loss: 18.1009 - mean_absolute_error: 18.1009 - mean_absolute_percentage_error: 324011364.2462 - val_loss: 17.6219 - val_mean_absolute_error: 17.6219 - val_mean_absolute_percentage_error: 316682581.9260
Epoch 2/3
2467080/2467080 [==============================] - 22s 9us/step - loss: 17.5316 - mean_absolute_error: 17.5316 - mean_absolute_percentage_error: 319777000.9549 - val_loss: 17.5389 - val_mean_absolute_error: 17.5389 - val_mean_absolute_percentage_error: 32948

2467080/2467080 [==============================] - 43s 17us/step - loss: 18.4680 - mean_absolute_error: 18.4680 - mean_absolute_percentage_error: 293020557.8762 - val_loss: 18.5249 - val_mean_absolute_error: 18.5249 - val_mean_absolute_percentage_error: 309825109.0637
Current epoch: [18.666378866538604, 18.488035686092196, 18.467993825410908] [298035328.4080734, 293431027.3025886, 293020557.8762494]
Train on 2467080 samples, validate on 274121 samples
Epoch 1/3
2467080/2467080 [==============================] - 45s 18us/step - loss: 12.4338 - mean_absolute_error: 12.4338 - mean_absolute_percentage_error: 47332498.1387 - val_loss: 12.6506 - val_mean_absolute_error: 12.6506 - val_mean_absolute_percentage_error: 10382004.1643
Epoch 2/3
2467080/2467080 [==============================] - 43s 17us/step - loss: 11.8129 - mean_absolute_error: 11.8129 - mean_absolute_percentage_error: 20837268.1513 - val_loss: 11.9690 - val_mean_absolute_error: 11.9690 - val_mean_absolute_percentage_error: 1091

2467080/2467080 [==============================] - 51s 21us/step - loss: 11.4899 - mean_absolute_error: 11.4899 - mean_absolute_percentage_error: 11887125.0828 - val_loss: 11.8824 - val_mean_absolute_error: 11.8824 - val_mean_absolute_percentage_error: 29993684.6233
Current epoch: [12.280888026943948, 11.62293499606839, 11.489941600042552] [40880751.38879365, 14288097.936330186, 11887125.082817005]
Train on 2467080 samples, validate on 274121 samples
Epoch 1/3
2467080/2467080 [==============================] - 50s 20us/step - loss: 18.6108 - mean_absolute_error: 18.6108 - mean_absolute_percentage_error: 283694850.6775 - val_loss: 18.3724 - val_mean_absolute_error: 18.3724 - val_mean_absolute_percentage_error: 279471941.8551
Epoch 2/3
2467080/2467080 [==============================] - 48s 19us/step - loss: 18.4335 - mean_absolute_error: 18.4335 - mean_absolute_percentage_error: 279768001.8813 - val_loss: 18.4372 - val_mean_absolute_error: 18.4372 - val_mean_absolute_percentage_error: 30

In [5]:
best_run

{'Dense': 2,
 'Dense_1': 2,
 'Dense_2': 1,
 'activation': 2,
 'activation_1': 0,
 'activation_2': 1,
 'batch_size': 0,
 'optimizer': 0}

In [6]:
best_model.evaluate()

ValueError: When using data tensors as input to a model, you should specify the `steps` argument.